In [17]:
corpus = {}
label = {}
label_index = {}
fname = ['train','dev','test']
for file_s in fname:
    fopen = open('extract_input/'+file_s+'.txt','r')
    _corpus = []
    _label = [[],[],[],[]]
    _label_index = []
    ind = 0
    for line in fopen:
        list_ = line.strip().split('\t')
        text,label1,label2,label3,label4 = list_[-5:]
        textlist = text.split('</s><s>')
        n = len(textlist)
        _corpus.extend(textlist)
        _label_index.extend([ind]*n)
        _label[0].append(label1)
        _label[1].append(label2)
        _label[2].append(label3)
        _label[3].append(label4)
        ind+=1
    fopen.close()
    corpus[file_s] = _corpus
    label[file_s] = _label
    label_index[file_s] = _label_index

In [19]:
import pandas as pd 
from pandas import Series, DataFrame
import numpy as np

from pythonrouge.pythonrouge import Pythonrouge


for file_s in fname:
    N = len(corpus[file_s])
    rouge_val = []
    for i in range(N):
        text = corpus[file_s][i]
        _index  = label_index[file_s][i]
        label_s = [[label[file_s][j][_index]] for j in range(4)]
        summary = [[text]]
        reference = [label_s]
        rouge = Pythonrouge(summary_file_exist=False,
                summary=summary, reference=reference,
                n_gram=2, ROUGE_SU4=True, ROUGE_L=False,
                recall_only=True, stemming=True, stopwords=True,
                word_level=True, length_limit=True, length=50,
                use_cf=False, cf=95, scoring_formula='best',
                resampling=True, samples=1000, favor=True, p=0.5)
        score = rouge.calc_score()
        rouge_val.append([score['ROUGE-1'], score['ROUGE-2'], score['ROUGE-SU4']])

        if (i+1)%500==0:
            print("%.2f %% has completed" %((i+1.0)/N*100))

    rouge_data = DataFrame(rouge_val, columns = ['ROUGE-1','ROUGE-2','ROUGE-SU4'])
    rouge_data.to_csv(file_s+'_bestrouge.csv', index = False)

5.51 % has completed
11.03 % has completed
16.54 % has completed
22.06 % has completed
27.57 % has completed
33.08 % has completed
38.60 % has completed
44.11 % has completed
49.63 % has completed
55.14 % has completed
60.65 % has completed
66.17 % has completed
71.68 % has completed
77.19 % has completed
82.71 % has completed
88.22 % has completed
93.74 % has completed
99.25 % has completed
51.44 % has completed
48.40 % has completed
96.81 % has completed


In [20]:
startpos = 0
for file_s in fname:
    N = len(corpus[file_s])
    rouge_data = DataFrame(rouge_val[startpos:startpos+N], columns = ['ROUGE-1','ROUGE-2','ROUGE-SU4'])
    rouge_data.to_csv(file_s+'_bestrouge.csv', index = False)
    startpos +=N